<a href="https://colab.research.google.com/github/mahdiardestani/solving_captcha/blob/main/captcha_transferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Input, MaxPool2D, BatchNormalization, GlobalAveragePooling2D, Dropout
from random import shuffle 
from sklearn.model_selection import train_test_split
from pathlib import Path
from sklearn import preprocessing
import cv2
import os
import glob
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!unzip CaptchaDataset.zip -d .> /dev/null

In [ ]:
images = []
labels = []
path = Path(r'/content/CaptchaDataset')
print('here')

for index, item in enumerate(path.glob('*/*')):
#     print('here2')    
    img = cv2.imread(str(item))
    img = cv2.resize(img,(32,32))
    #Normalize image
    img = img/255.0
    images.append(img)
    label = (str(item)).split("/")[-1].split('_')[0]
    labels.append(label)

here


#  Test set

In [ ]:
!unzip subtest_data.zip -d . > /dev/null

In [ ]:
images_test = []
labels_test = []
for index, item in enumerate(glob.glob('subtest_data/*.jpg')):
#     print('here2')    
    img = cv2.imread(str(item))
    img = cv2.resize(img,(32,32))
    #Normalize image
    img = img/255.0
    images_test.append(img)
    label = (str(item)).split("/")[-1].split('_')[0]
    labels_test.append(label)

In [ ]:
print(len(images_test), len(labels_test))
print(labels_test)

153 153
['Y', 'H', 'W', 'A', 'X', 'T', 'V', 'A', 'N', 'Y', '9', 'N', 'K', 'Y', 'Y', '4', 'H', 'A', 'A', '3', 'P', 'V', 'X', 'T', 'U', 'H', 'T', 'E', '8', 'C', 'C', 'S', 'D', 'S', 'V', '6', 'C', 'U', 'P', 'H', 'D', 'R', 'N', 'D', 'U', 'T', 'X', 'A', 'U', 'W', 'C', 'E', 'W', '4', '8', '4', 'U', 'W', 'M', 'D', 'N', 'V', 'A', 'W', '6', 'S', '4', 'V', 'D', 'P', 'R', 'P', 'A', '5', 'H', 'E', '3', 'C', '4', 'C', 'X', 'A', '3', 'K', 'Y', '4', 'A', '8', '5', 'V', '4', 'W', 'A', 'P', 'P', 'Y', 'X', 'Y', 'K', '4', '5', 'H', '9', '4', 'H', 'J', 'E', 'W', 'D', 'D', 'X', '8', 'S', '3', 'S', 'D', 'T', 'R', 'J', 'H', 'X', 'B', 'U', 'U', 'R', 'D', 'U', 'S', '6', 'E', '5', 'R', '5', 'T', '6', 'H', 'T', '5', 'B', '8', 'S', 'P', 'D', 'K', '9', 'M', '9', 'C', '9', 'T', 'D', 'W', '8']


In [ ]:
images_test = np.array(images_test)
# multilabel encoding 
mlb_test = preprocessing.LabelBinarizer()
labels_test = mlb_test.fit_transform(labels_test)

In [ ]:
print(images_test.shape, labels_test.shape)

(153, 32, 32, 3) (153, 25)


In [ ]:
#find len of labels with unique value in set
labels_set = list(set(labels))
no_classes = len(labels_set)
# print(len(label_set))

In [ ]:
images = np.array(images)
# multilabel encoding 
mlb = preprocessing.LabelBinarizer()
labels = mlb.fit_transform(labels)
# print(type(labels))
mlb.classes_
Xtrain, Xtest, ytrain, ytest = train_test_split(images, labels, test_size = 0.2)
# print(len(Xtrain))
print("shape of Xtrain: ", Xtrain.shape, end = '\n')
print("shape of Xtest: ", Xtest.shape, end = '\n')
# print(Xtrain.shape[1:])
# print(type(ytrain))
print("shape of ytrain:", ytrain.shape,  end = '\n')
print("shape of ytest:", ytest.shape,  end = '\n')
# print(type(ytrain_arr))
print(no_classes)

shape of Xtrain:  (14782, 32, 32, 3)
shape of Xtest:  (3696, 32, 32, 3)
shape of ytrain: (14782, 25)
shape of ytest: (3696, 25)
25


In [ ]:
def generateModel(no_classes):
    model = Sequential()
    model.add(Conv2D(32, 3, padding='same', activation='selu', input_shape=Xtrain.shape[1:]))
    model.add(BatchNormalization())
    model.add(MaxPool2D())
    model.add(Dropout(.20))
    
    model.add(Conv2D(64, 3, padding='same', activation='selu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D())

    model.add(Conv2D(128, 3, padding='same', activation='selu', input_shape=Xtrain.shape[1:]))
    model.add(BatchNormalization())
    model.add(MaxPool2D())
    model.add(Dropout(.20))
    
    model.add(GlobalAveragePooling2D())
    
    model.add(Dense(no_classes, activation='softmax'))

    model.summary()
    return model

model = generateModel(no_classes)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 batch_normalization_2 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 16, 16, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 batch_normalization_3 (Batc  (None, 16, 16, 64)      

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
xtrain_arr = np.asarray(Xtrain).astype('float32')
print(xtrain_arr.shape[0:-1])
xtest_arr = np.asarray(Xtest).astype('float32')
print(xtest_arr.shape)
ytrain_arr = np.asarray(ytrain).astype('float32')
print(ytrain_arr.shape)
ytest_arr = np.asarray(ytest).astype('float32')
print(ytest_arr.shape)
result = model.fit(xtrain_arr, ytrain_arr, batch_size = 32, epochs = 20, validation_data = (xtest_arr, ytest_arr))

(14782, 32, 32)
(3696, 32, 32, 3)
(14782, 25)
(3696, 25)
Epoch 1/20
462/462 [==============================] - 10s 6ms/step - loss: 0.3483 - acc: 0.8921 - val_loss: 0.5629 - val_acc: 0.8382
Epoch 2/20
462/462 [==============================] - 2s 5ms/step - loss: 0.3153 - acc: 0.9039 - val_loss: 0.5855 - val_acc: 0.8350
Epoch 3/20
462/462 [==============================] - 2s 5ms/step - loss: 0.2970 - acc: 0.9078 - val_loss: 0.6095 - val_acc: 0.8369
Epoch 4/20
462/462 [==============================] - 2s 5ms/step - loss: 0.2771 - acc: 0.9158 - val_loss: 0.5846 - val_acc: 0.8406
Epoch 5/20
462/462 [==============================] - 2s 5ms/step - loss: 0.2557 - acc: 0.9209 - val_loss: 0.5479 - val_acc: 0.8460
Epoch 6/20
462/462 [==============================] - 2s 5ms/step - loss: 0.2540 - acc: 0.9188 - val_loss: 0.5705 - val_acc: 0.8393
Epoch 7/20
462/462 [==============================] - 2s 5ms/step - loss: 0.2341 - acc: 0.9280 - val_loss: 0.5533 - val_acc: 0.8458
Epoch 8/20
462/462

In [ ]:
model.evaluate(images_test, labels_test)

5/5 [==============================] - 0s 20ms/step - loss: 0.6446 - acc: 0.8235


[0.6445950269699097, 0.8235294222831726]

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

#Loading Model
base_model = VGG16(weights="imagenet", include_top=False,input_shape=Xtrain.shape[1:])
base_model.trainable = False

Xtrain_vgg = preprocess_input(Xtrain)
Xtest_vgg = preprocess_input(Xtest)
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(no_classes, activation='softmax')(x)
# base_model.summary()

model_vgg = Model(inputs=base_model.input, outputs=x)
model_vgg.summary()

58900480/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                              

In [ ]:
model_vgg.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
model_vgg.fit(xtrain_arr, ytrain_arr, batch_size = 32, epochs = 20, validation_data = (xtest_arr, ytest_arr))

Epoch 1/20
462/462 [==============================] - 7s 14ms/step - loss: 2.6121 - acc: 0.2441 - val_loss: 2.2921 - val_acc: 0.3304
Epoch 2/20
462/462 [==============================] - 6s 12ms/step - loss: 2.1490 - acc: 0.3775 - val_loss: 2.0783 - val_acc: 0.3858
Epoch 3/20
462/462 [==============================] - 6s 12ms/step - loss: 1.9766 - acc: 0.4262 - val_loss: 1.9634 - val_acc: 0.4183
Epoch 4/20
462/462 [==============================] - 6s 12ms/step - loss: 1.8678 - acc: 0.4554 - val_loss: 1.8836 - val_acc: 0.4356
Epoch 5/20
462/462 [==============================] - 6s 13ms/step - loss: 1.7895 - acc: 0.4778 - val_loss: 1.8314 - val_acc: 0.4535
Epoch 6/20
462/462 [==============================] - 6s 12ms/step - loss: 1.7303 - acc: 0.4930 - val_loss: 1.7884 - val_acc: 0.4637
Epoch 7/20
462/462 [==============================] - 6s 12ms/step - loss: 1.6798 - acc: 0.5083 - val_loss: 1.7539 - val_acc: 0.4751
Epoch 8/20
462/462 [==============================] - 6s 13ms/step - 

In [ ]:
model_vgg.evaluate(images_test, labels_test)

5/5 [==============================] - 1s 101ms/step - loss: 1.5678 - acc: 0.5294


[1.5677504539489746, 0.529411792755127]